<a href="https://colab.research.google.com/github/drfperez/DeepPurpose/blob/main/DeepPurpose8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip install pubchempy
!pip install DeepPurpose
!pip install rdkit
!pip install git+https://github.com/bp-kelley/descriptastorus
!pip install pandas-flavor
!pip install gradio
!pip install Bio

# Import the libraries
import numpy as np
import pandas as pd
from DeepPurpose import utils
from DeepPurpose import DTI as models
import gradio as gr
import requests
from Bio import SeqIO
from io import BytesIO
import pubchempy as pc

# Load the pretrained models for different datasets
model_binding = models.model_pretrained(model='MPNN_CNN_BindingDB')
model_kiba = models.model_pretrained(model='MPNN_CNN_KIBA')
model_davis = models.model_pretrained(model='MPNN_CNN_DAVIS')

def fetch_pdb_sequence(pdb_code):
    try:
        # Fetch the PDB file from RCSB PDB website
        pdb_url = f"https://files.rcsb.org/download/{pdb_code.lower()}.pdb"
        response = requests.get(pdb_url)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the response content as a PDB file
            pdb_content = response.content
            pdb_file = BytesIO(pdb_content)

            # Parse the PDB file and extract the target sequence
            target_sequence = ""
            for structure in SeqIO.parse(pdb_file, "pdb-seqres"):
                target_sequence += str(structure.seq)
            return target_sequence
        else:
            return f"Error: PDB code {pdb_code} not found."
    except Exception as e:
        return f"Error fetching sequence for PDB code {pdb_code}: {str(e)}"

def get_smiles_from_name(chemical_name):
    try:
        compound = pc.get_compounds(chemical_name, 'name')
        if compound:
            return compound[0].canonical_smiles
        else:
            return None
    except Exception as e:
        return f"Error fetching SMILES for {chemical_name}: {str(e)}"

def DTI_pred(data, drug_type, drug_input, target_type, target_input):
    """
    This function predicts the drug-target interaction affinity for a given dataset, drug, and target.
    Parameters:
    - data: the name of the dataset to use for the prediction, one of ['BindingDB', 'DAVIS', 'KIBA']
    - drug_type: the type of drug input, either 'SMILES' or 'Chemical Names'
    - drug_input: a string of one or more drugs in SMILES format or chemical names, separated by comma
    - target_type: the type of target input, either 'PDB Code', 'Upload PDB File', or 'Sequence'
    - target_input: the target input (either PDB code, uploaded PDB file, or sequence)
    Returns:
    - a string of the predicted affinities for each drug, separated by comma
    """
    try:
        if target_type == 'PDB Code':
            target = fetch_pdb_sequence(target_input)
        elif target_type == 'Upload PDB File':
            if target_input is None:
                return "Error: No file uploaded."
            pdb_file = BytesIO(target_input.read())
            target_sequence = ""
            for structure in SeqIO.parse(pdb_file, "pdb-seqres"):
                target_sequence += str(structure.seq)
            target = target_sequence
        elif target_type == 'Sequence':
            target = target_input

        if drug_type == 'SMILES':
            drugs = drug_input.split(",")
        elif drug_type == 'Chemical Names':
            drugs = [get_smiles_from_name(name.strip()) for name in drug_input.split(",")]
            drugs = [smiles for smiles in drugs if smiles]

        if data == 'BindingDB':
            model = model_binding
        elif data == 'KIBA':
            model = model_kiba
        elif data == 'DAVIS':
            model = model_davis

        predictions = []

        for drug in drugs:
            X_pred = utils.data_process(X_drug=[drug], X_target=[target], y=[0],
                                        drug_encoding='MPNN', target_encoding='CNN',
                                        split_method='no_split')

            y_pred = model.predict(X_pred)

            predictions.append(str(y_pred[0]))

        return ", ".join(predictions)
    except Exception as e:
        return f"Error predicting drug-target interaction: {str(e)}"

# Create a Gradio interface with the inputs, outputs, and title
interface = gr.Interface(fn=DTI_pred,
                         inputs=[gr.Dropdown(label="Training Dataset", choices=['BindingDB', 'DAVIS', 'KIBA']),
                                 gr.Dropdown(label="Drug Input Type", choices=['SMILES', 'Chemical Names']),
                                 gr.Textbox(lines=5, label="Drug Input (separated by comma)"),
                                 gr.Radio(label="Target Input Type", choices=['Execute PDB Code', 'Upload PDB File']),
                                 gr.File(label="Upload PDB File", type="binary")],
                         outputs=gr.Textbox(label="Predicted Affinities"),
                         title="Drug-Target Interaction Prediction with DeepPurpose")

# Launch the interface
interface.launch()

  Cloning https://github.com/bp-kelley/descriptastorus to /tmp/pip-req-build-qopd80mq
  Running command git clone --filter=blob:none --quiet https://github.com/bp-kelley/descriptastorus /tmp/pip-req-build-qopd80mq
  Resolved https://github.com/bp-kelley/descriptastorus to commit da9760932ab9a78b116bc697795dd9e1798f087a
  Preparing metadata (setup.py) ... done
